In [17]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings

import requests 
import json

warnings.filterwarnings('ignore')
%matplotlib inline

import dtale

In [18]:
df = pd.read_csv("angers_bus_tram_json_merge.csv", sep=";", encoding="utf-8")
df.sample(5)

,Unnamed: 0,datasetid,recordid,record_timestamp,fields.iddesserte,fields.idvh,fields.numarret,fields.etat,fields.novh,fields.nomligne,...,fields.coordonnees,fields.idarret,fields.cap,fields.idparcours,fields.sv,fields.y,fields.x,fields.ts_maj,geometry.type,geometry.coordinates
193436,193436,bus-tram-position-tr,a777f1ace4019b5f775a24a70db74c8a946649ce,2019-09-25T03:59:58.756000+00:00,268689666,268436074,1649.0,LIGN,618,MURS ERIGNE <> ADEZIERE SALETTE,...,"[47.496458, -0.56846502]",1523654,291,268689664.0,0321,2281473,381172,2019-09-25T03:59:57+00:00,Point,"[-0.56846502, 47.496458]"
248057,248057,bus-tram-position-tr,231b81cccbb37cab945625d82e87f508a4974653,2019-09-30T18:44:58.400000+00:00,268708152,268436148,15981.0,LIGN,692,BEAUCOUZE <> ST BARTHELEMY,...,"[47.459903, -0.4866675]",1533238,123,268708096.0,0403,2277187,387185,2019-09-30T18:44:58+00:00,Point,"[-0.4866675, 47.459903]"
190215,190215,bus-tram-position-tr,d1944ceda2e6d10638f043e917ada83be2473f5f,2019-10-25T08:14:55.682000+00:00,268507945,268436077,837.0,LIGN,621,BELLE BEILLE <> MONPLAISIR,...,"[47.487663, -0.53203895]",1580924,23,268507904.0,0120,2280395,383879,2019-10-25T08:14:55+00:00,Point,"[-0.53203895, 47.487663]"
139728,139728,bus-tram-position-tr,6649fd4dfef1b561ba530e1bc102b820b6dbe42c,2019-10-15T04:44:58.034000+00:00,269442305,268436470,16020.0,LIGN,1014,ARDENNE <> ROSERAIE,...,"[47.495446, -0.56825044]",1562223,56,269442304.0,A13,2281360,381184,2019-10-15T04:44:57+00:00,Point,"[-0.56825044, 47.495446]"
204934,204934,bus-tram-position-tr,970584f6ee3dc808db238005d8da0cac59de5961,2019-09-25T08:29:52.971000+00:00,268507924,268436067,913399.0,LIGN,611,BELLE BEILLE <> MONPLAISIR,...,"[47.467996, -0.55917372]",1525353,192,268507904.0,0103,2278285,381755,2019-09-25T08:29:52+00:00,Point,"[-0.55917372, 47.467996]"


In [19]:
df = (df
        .drop(columns=["Unnamed: 0", "datasetid", "recordid", "geometry.type"], errors='ignore')
        .astype({"record_timestamp": "datetime64", "fields.ts_maj": "datetime64", "fields.harret" : "datetime64", "fields.coordonnees" : "string"})
        .assign(ecart_horodatage = lambda x: x["record_timestamp"] - x["fields.ts_maj"])
)


In [20]:
nomsColonnes = ['horodatage', 
                'identifiant_SAE_de_desserte',
                'identifiant_du_vehicule',
                'numero_Timeo_de_l_arret',
                'etat_SAE_du_vehicule', 
                'numero_de_parc_du_vehicule',
                'nom_de_la_ligne',
                'Heure_estimee_de_passage_a_L_arret', 
                'modele_du_vehicule',
                'identifiant_SAE_de_ligne',
                'ecart_horaire_en_secondes',
                'destination',
                'nom_de l_arret', 
                'mne_de_l_arret',
                'mnemo_de_la_ligne',
                 'coordonnees_GPS_WG84', 
                'identifiant_SAE_de_l_arret',
                'cap_du_vehicule_en_degres', 
                'identifiant_SAE_du_parcours',
                'service_voiture', 
                'coordonnees_GPS_X',
                'coordonnees_GPS_Y', 
                'horodatage_maj',
                'cordonnees_bus_geometrie', 
                'ecart_horodatage']


df.columns = nomsColonnes

In [21]:
# index des lignes dupliquées (on drop les unhashable)
duplicateRowsDF = df[df.drop(columns=["cordonnees_bus_geometrie", "coordonnees_GPS_WG84"]).duplicated()]

# drop des lignes dupliquées par index
df = df.drop(duplicateRowsDF.index, axis=0)
df.shape

(728442, 25)

In [22]:
df.isna().sum()

df = df.dropna()

df.isna().sum().sum()

0

In [23]:
q1 = df["ecart_horaire_en_secondes"].quantile(0.05)
q3 = df["ecart_horaire_en_secondes"].quantile(0.95)
iqr = q3 - q1

df["ecart_horaire_en_secondes"] = df["ecart_horaire_en_secondes"].clip(lower=q1, upper=q3)

In [24]:
df[['latitude', 'longitude']] = df['coordonnees_GPS_WG84'].str.split(',', expand=True)

cols = ['latitude', 'longitude']
for col in cols :
    df[col] = df[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

df["year"] = df["horodatage"].dt.year
df["month"] = df["horodatage"].dt.month
df["day"] =  df["horodatage"].dt.day
df["hours"] = df["horodatage"].dt.hour
df["date"] = pd.to_datetime(df[["year", "month", "day"]])


df["jour_semaine"] = df["date"].dt.day_name().map({"Monday": "Lundi", "Tuesday": "Mardi", "Wednesday": "Mercredi", "Thursday": "Jeudi", "Friday": "Vendredi", "Saturday": "Samedi", "Sunday": "Dimanche"})
df.drop(columns=["date"], inplace=True)

In [25]:
df.columns

Index(['horodatage', 'identifiant_SAE_de_desserte', 'identifiant_du_vehicule',
       'numero_Timeo_de_l_arret', 'etat_SAE_du_vehicule',
       'numero_de_parc_du_vehicule', 'nom_de_la_ligne',
       'Heure_estimee_de_passage_a_L_arret', 'modele_du_vehicule',
       'identifiant_SAE_de_ligne', 'ecart_horaire_en_secondes', 'destination',
       'nom_de l_arret', 'mne_de_l_arret', 'mnemo_de_la_ligne',
       'coordonnees_GPS_WG84', 'identifiant_SAE_de_l_arret',
       'cap_du_vehicule_en_degres', 'identifiant_SAE_du_parcours',
       'service_voiture', 'coordonnees_GPS_X', 'coordonnees_GPS_Y',
       'horodatage_maj', 'cordonnees_bus_geometrie', 'ecart_horodatage',
       'latitude', 'longitude', 'year', 'month', 'day', 'hours',
       'jour_semaine'],
      dtype='object')

In [26]:
# on transforme l'heure estimee de passage à l 'arret  : 
df["year_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.year
df["month_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.month
df["day_estimation_passage_arret"] =  df["Heure_estimee_de_passage_a_L_arret"].dt.day
df["hours_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.hour
df["minutes_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.minute
df["secondes_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.second
df.drop(columns=["Heure_estimee_de_passage_a_L_arret"], inplace=True)



In [27]:
# dummy 
df = pd.get_dummies(df)

MemoryError: Unable to allocate 89.8 GiB for an array with shape (133076, 724202) and data type uint8